In [1]:
import pprint
import numpy as np
import pandas as pd
from dateutil import parser
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay

In [2]:
%matplotlib inline

## Global variables

In [3]:
DATA_DIR = '../data'
IS_BUSINESS_DAY = BDay().onOffset

### Reading and basic preprocessing of data

In [4]:
xau_cny = pd.read_csv(f'{DATA_DIR}/XAU_CNY Historical Data.csv')
xau_cny = xau_cny.iloc[:, [0, 1]]

xau_cny['Price'] = xau_cny['Price'].str.replace(',', '').astype(float)

xau_cny['Date'] = pd.to_datetime(xau_cny['Date'])
xau_cny = xau_cny.sort_values(by=['Date']).reset_index(drop=True)

print(f'There are {xau_cny.shape[0]} values for XAU/CNY\n')
print('Head values \n', xau_cny.head(10), '\n')
print('Tail values\n', xau_cny.tail(10))

There are 1341 values for XAU/CNY

Head values 
         Date    Price
0 2015-09-14  7062.32
1 2015-09-15  7040.48
2 2015-09-16  7131.97
3 2015-09-17  7203.44
4 2015-09-18  7249.60
5 2015-09-21  7218.10
6 2015-09-22  7170.84
7 2015-09-23  7215.85
8 2015-09-24  7366.36
9 2015-09-25  7306.23 

Tail values
            Date     Price
1331 2020-09-02  13281.11
1332 2020-09-03  13209.95
1333 2020-09-04  13227.38
1334 2020-09-07  13173.86
1335 2020-09-08  13216.03
1336 2020-09-09  13292.67
1337 2020-09-10  13298.35
1338 2020-09-11  13253.90
1339 2020-09-13  13277.80
1340 2020-09-14  13286.51


In [5]:
usd_cny = pd.read_csv(f'{DATA_DIR}/USD_CNY Historical Data.csv')
usd_cny = usd_cny.iloc[:, [0, 1]]

usd_cny['Date'] = pd.to_datetime(usd_cny['Date'])
usd_cny = usd_cny.sort_values(by=['Date']).reset_index(drop=True)

print(f'There are {usd_cny.shape[0]} values for USD/CNY\n')
print('Head values \n', usd_cny.head(10), '\n')
print('Tail values\n', xau_cny.tail(10))

There are 1301 values for USD/CNY

Head values 
         Date   Price
0 2015-09-14  6.3679
1 2015-09-15  6.3700
2 2015-09-16  6.3709
3 2015-09-17  6.3660
4 2015-09-18  6.3643
5 2015-09-21  6.3691
6 2015-09-22  6.3760
7 2015-09-23  6.3837
8 2015-09-24  6.3833
9 2015-09-25  6.3741 

Tail values
            Date     Price
1331 2020-09-02  13281.11
1332 2020-09-03  13209.95
1333 2020-09-04  13227.38
1334 2020-09-07  13173.86
1335 2020-09-08  13216.03
1336 2020-09-09  13292.67
1337 2020-09-10  13298.35
1338 2020-09-11  13253.90
1339 2020-09-13  13277.80
1340 2020-09-14  13286.51


In [6]:
# Remove non-business days.
xau_cny_valid_dates = xau_cny['Date'].map(IS_BUSINESS_DAY)
usd_cny_valid_dates = usd_cny['Date'].map(IS_BUSINESS_DAY)

# Select business dates data.
xau_cny = xau_cny[xau_cny_valid_dates].reset_index(drop=True)
usd_cny = usd_cny[usd_cny_valid_dates].reset_index(drop=True)

print('Shape after preprocessing is :')
print('For XAU/CNY', xau_cny.shape)
print('For USD/CNY', usd_cny.shape)

Shape after preprocessing is :
For XAU/CNY (1305, 2)
For USD/CNY (1301, 2)


/home/yash/.local/lib/python3.8/site-packages/pandas/core/base.py:1160: FutureWarning: onOffset is a deprecated, use is_on_offset instead
  new_values = map_f(values, mapper)


In [ ]:
if usd_cny.shape[0] > xau_cny.shape[0]:
    A = usd_cny
    B = xau_cny
else:
    B = usd_cny
    A = xau_cny

In [ ]:
START_CONFIG = {
    'vault': 2000,
    'a': 1000,
    'b': 1000,
    'abp': 0,
    'bbp': 0
}

pp = pprint.PrettyPrinter(depth=6)

In [ ]:
START_CONFIG['abp'] = START_CONFIG.get('a') / A.loc[0, 'Price']

In [ ]:
START_CONFIG['bbp'] = START_CONFIG.get('b') / B.loc[0, 'Price']

In [ ]:
START_CONFIG

In [ ]:
a_prev = A.iloc[0, :]
b_prev = B.iloc[0, :]

for i in A.index.to_list():
    # Take the latest value or the last value.
    if i < A.shape[0]: a = A.loc[i, :]
    else: a = A.loc[A.index.to_list()[-1], :]
    
    if i < B.shape[0]: b = B.loc[i, :]
    else: b = B.loc[B.index.to_list()[-1], :]

    if a_prev.Price == a.Price and b_prev.Price == b.Price: continue
    
    # Get the new CNY values.
    new_config_a = START_CONFIG['abp'] * a.Price
    new_config_b = START_CONFIG['bbp'] * b.Price
    
    # Calculate the change in indiviaul CNY values.
    START_CONFIG['a'] = (new_config_a)
    START_CONFIG['b'] = (new_config_b)
    
    # START_CONFIG['a'] = START_CONFIG['abp'] * a.Price
    # START_CONFIG['b'] = START_CONFIG['bbp'] * b.Price
    
    # Calculate the diff between the 2 assets.
    diff = START_CONFIG['a'] - START_CONFIG['b']
    half_diff = diff / 2
    
    # Rebalance the assets.
    if diff == 0: continue
    else:
        START_CONFIG['a'] -= half_diff
        START_CONFIG['b'] += half_diff
        
    # Recalculate the buying power based on current price & current asset values.
    START_CONFIG['abp'] = START_CONFIG.get('a') / a.Price
    START_CONFIG['bbp'] = START_CONFIG.get('b') / b.Price
    
    # Recalculate the vault balance.
    START_CONFIG['vault'] = START_CONFIG.get('a') + START_CONFIG.get('b')
    
    print()
    pp.pprint({'Vault': START_CONFIG, 'Time': a.Date})
    print()
    
    if (START_CONFIG['vault'] - 2000) / 2000 <= -0.09: print('Breaking'); break
    
    a_prev, b_prev = a, b